## This file implements translation evaluations using a modified GEMBA prompt system for the test dataset.

### Install Libraries

In [1]:
%pip install pandas
%pip install langchain
%pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Imports

In [44]:
import os
import ast
import pandas as pd
from tqdm import tqdm

# Langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### Globals

In [3]:
LLM_MODEL_NAME="gpt-3.5-turbo"

### Utility function to set up the OpenAI API

In [4]:
def load_openai_key():
    try:
        # Open and read the entire content of the file
        with open("openai-key.txt", 'r') as file:
            contents = file.read()
        
        return contents
    
    except FileNotFoundError:
        print(f"The file does not exist.")
        return None
    except Exception as e:
        # Handle other potential exceptions (e.g., permission errors)
        print(f"An error occurred while reading the file: {str(e)}")
        return None

openai_key = load_openai_key()
os.environ["OPENAI_API_KEY"] = openai_key

chat = ChatOpenAI(model="gpt-3.5-turbo")

### Prompt definition

In [76]:
system = """\
You are a helpful evaluator of the quality of translations.
Score the following translations from English to Portuguese on a continuous scale from 0 to 100, 
where a score of zero means "no meaning preserved" and score of one hundred means
"perfect meaning and grammar".
"""

human = """\
En: {source_seg}
Pt 1: "{target_seg1}" 
Pt 2: "{target_seg2}"
Pt 3: "{target_seg3}"
Reply with only the number scores of your evaluation, in a python list:
"""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

### Initial test

In [73]:
chain = prompt | chat
response = chain.invoke(input={"source_seg": "There are more things between heaven and earth",
                    "target_seg1": "Existem mais coisas entre o céu e a terra",
                    "target_seg2": "Existem coisas entre o céu e a terra",
                    "target_seg3": "Existem coisas entre o paraíso e a terra"})

In [75]:
print(response.content)

[90, 80, 70]


In [74]:
print(response.response_metadata.get('token_usage').get('total_tokens'))

155


In [46]:
print(response.content)

[90, 75, 60]


In [47]:
actual_list = ast.literal_eval(response.content)

In [49]:
actual_list[0]

90

### Translation Datasets for comparisons (TEST)

In [8]:
df_tst = pd.read_csv('data/tst.csv')
df_tst = df_tst.drop(columns=['Unnamed: 0'])

In [9]:
df_tst

,context,question,answerA,answerB,answerC,correct
0,bailey was a nice person so she called the fam...,What will happen to Others?,talk to the family,hate bailey,thank bailey,C
1,Alex found a ring on the beach and tried to fi...,What will the person who lost want to do next?,throw it in the ocean,turn it into lost and found,leave the beach,B
2,Quinn player with remy swinging her around and...,What will Quinn want to do next?,be friendly with Remy,have the trust of Remy,Put Remy down,C
3,Casey was popular throughout high school. Case...,How would you describe Casey?,a lazy person,casey who has a star of football player,a social person,C
4,Alex bumped the table that the coffee was plac...,What will happen to Riley?,continue to drink the coffee,spill the coffee,have to buy a refill,B
...,...,...,...,...,...,...
2219,Bailey spent the whole weekend at Alex's house...,How would Bailey feel afterwards?,stressed,serene,agitated,B
2220,"Jesse ate, drank, and talked with his friends ...",What will Jesse want to do next?,go home next,order more food next,drink all he wanted,A
2221,Alex watched cartoons and had a snack after sc...,Why did Alex do this?,do homework,relax,have to think about school,B
2222,Quinn name the films Jordan is most proud of a...,How would you describe Quinn?,proud of Jordan's progress,thankful to be a part of Jordan's journey,a writer,A


In [10]:
df_tst1_pt = pd.read_csv('translated/marian_mt/tst_pt.csv')
df_tst2_pt = pd.read_csv('translated/unicamp_t5/tst_pt.csv')
df_tst3_pt = pd.read_csv('translated/nllb_1.3b/tst_pt.csv')

# clean up data
df_tst1_pt = df_tst1_pt.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
df_tst2_pt = df_tst2_pt.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'correct'])
df_tst3_pt = df_tst3_pt.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'correct'])

# Rename columns
df_tst1_pt = df_tst1_pt.rename(columns={'context': 'context_1', 'question': 'question_1',
                                  'answerA': 'answerA_1', 'answerB': 'answerB_1',
                                  'answerC': 'answerC_1'})
df_tst2_pt = df_tst2_pt.rename(columns={'context': 'context_2', 'question': 'question_2',
                                  'answerA': 'answerA_2', 'answerB': 'answerB_2',
                                  'answerC': 'answerC_2'})
df_tst3_pt = df_tst3_pt.rename(columns={'context': 'context_3', 'question': 'question_3',
                                  'answerA': 'answerA_3', 'answerB': 'answerB_3',
                                  'answerC': 'answerC_3'})
# Merge the three dataframes
df_tst_pt = pd.concat((df_tst1_pt, df_tst2_pt, df_tst3_pt), axis=1)
df_tst_pt = df_tst_pt[['context_1', 'context_2', 'context_3',
                 'question_1', 'question_2', 'question_3',
                 'answerA_1', 'answerA_2', 'answerA_3',
                 'answerB_1', 'answerB_2', 'answerB_3',
                 'answerC_1', 'answerC_2', 'answerC_3',
                 'correct']]

In [11]:
df_tst_pt

,context_1,context_2,context_3,question_1,question_2,question_3,answerA_1,answerA_2,answerA_3,answerB_1,answerB_2,answerB_3,answerC_1,answerC_2,answerC_3,correct
0,"O bailey era uma pessoa simpática, por isso ch...","bailey foi uma pessoa bonita, assim ela chamou...","A Bailey era uma pessoa simpática, por isso re...",O que acontecerá com os outros?,O que acontecerá com os outros?,O que acontecerá aos Outros?,Fale com a família,conversar com a família,falar com a família,Odeia o bailey,baile de ódio,Odeio Bailey .,Graças a Deus!,agradecer bailey,Obrigado Bailey .,C
1,Alex encontrou um anel na praia e tentou encon...,Alex encontrou um anel na praia e tentou encon...,Alex encontrou um anel na praia e tentou desco...,O que a pessoa que perdeu vai querer fazer a s...,O que a pessoa que perdeu quer fazer a seguir?,O que é que a pessoa que perdeu vai querer faz...,Jogue-o no oceano,jogar no oceano,Atira-o para o oceano .,transformá-lo em perdido e encontrado,a transformar em perdido e encontrado,"Transforme-o em ""perdidos e encontrados"".",Sair da praia,folgue a praia,Deixe a praia,B
2,Quinn jogador com remy balançando ela ao redor...,Quinn jogadora com remessa girando em torno de...,A Quinn tocava com o Remy a balançá-la e ela r...,O que Quinn vai querer fazer a seguir?,O que queremos Quinn?,O que quer o Quinn fazer a seguir?,Seja amigável com Remy,ser amigo de Remy,Seja simpático com o Remy.,Tenha a confiança de Remy,ter a confiança de Remy,Tenho a confiança do Remy.,Põe o Remy no chão.,Coloque Remy,Larga o Remy.,C
3,Casey era popular em todo o ensino médio. Case...,"A casey era popular durante o ensino médio, se...","O Casey era popular no liceu, era um jogador d...",Como você descreveria Casey?,Como descreveria Casey?,Como descreveria o Casey?,uma pessoa preguiçosa,uma pessoa preguiça,uma pessoa preguiçosa,Casey que tem uma estrela do jogador de futebol,casey que tem uma estrela de jogador de futebol,Casey que tem uma estrela de futebol,uma pessoa social,uma pessoa social,pessoa social,C
4,Alex bateu na mesa em que o café foi colocado.,Alex bompeu a mesa que o café foi colocado.,Alex bateu na mesa onde o café estava colocado.,O que vai acontecer com Riley?,O que acontecerá com Riley?,O que vai acontecer ao Riley?,Continue a beber o café,continuar a beber o café,Continua a beber o café.,Derramem o café,derramar o café,derramar o café,tem que comprar um refil,ter que comprar um reenchimento,Tenho de comprar um recheio.,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219,Bailey passou o fim de semana inteiro na casa ...,Bailey permaneceu o final de semana na casa de...,A Bailey passou o fim de semana inteiro em cas...,Como Bailey se sentiria depois?,Como Bailey se sentiria depois?,Como é que a Bailey se sentiria depois?,estressada,stress,estressado,serena,sereno,sereno,agitado,agitação,agitado,B
2220,"Jesse comeu, bebeu e conversou com seus amigos...","Jesse come, bebe e conversa com os amigos ante...","Jesse comeu, bebeu e conversou com os amigos a...",O que Jesse vai querer fazer a seguir?,O que Jesse quer fazer em seguida?,O que é que o Jesse vai querer fazer a seguir?,ir para casa a seguir,ir para casa seguinte,Vai para casa a seguir.,pedir mais comida a seguir,án án án án án án án án án,Pede mais comida a seguir.,Beba tudo o que ele queria,beba tudo o que ele quisesse,Beba o que quisesse.,A
2221,Alex assistiu desenhos animados e tomou um lan...,Alex assistiu a cartoons e fez lanche após a e...,Alex via desenhos animados e comeu um lanche d...,Porque é que o Alex fez isto?,Por que o Alex fez isso?,Porque é que a Alex fez isto?,fazer lição de casa,fazer tarefas domésticas,Fazer lições de casa,relaxar,lúpus,Relaxa .,É preciso pensar na escola,tem que pensar a escola,Tenho de pensar na escola.,B
2222,Quinn nomeia os filmes que Jordan está mais or...,Quinn nomeia os filmes Jordan é mais orgulhado...,"Quinn, diz-me os filmes de que o Jordan está m...",Como você descreveria Quinn?,Como descreveria Quinn?,Como 

In [93]:
def create_sequences(i):

        sentence_list = ['context', 'question', 'answerA', 'answerB', 'answerC']
        english_str=""
        for sentence in sentence_list:
                english_str += str(df_tst.loc[i, sentence])
                english_str += "\n"

        portuguese_1=""
        sentence_list = ['context_1', 'question_1', 'answerA_1', 'answerB_1', 'answerC_1']
        for sentence in sentence_list:
                portuguese_1 += str(df_tst_pt.loc[i, sentence])
                portuguese_1 += "\n"

        portuguese_2=""
        sentence_list = ['context_2', 'question_2', 'answerA_2', 'answerB_2', 'answerC_2']
        for sentence in sentence_list:
                portuguese_2 += str(df_tst_pt.loc[i, sentence])
                portuguese_2 += "\n"

        portuguese_3=""
        sentence_list = ['context_3', 'question_3', 'answerA_3', 'answerB_3', 'answerC_3']
        for sentence in sentence_list:
                portuguese_3 += str(df_tst_pt.loc[i, sentence])
                portuguese_3 += "\n"

        return english_str, portuguese_1, portuguese_2, portuguese_3

In [94]:
def get_best_translations(df_src_lang, df_tgt_lang):

    total_tokens = 0

    df_output = df_src_lang.copy()
    df_output.loc[:, ['context', 'question', 'answerA', 'answerB', 'answerC']] = None
    df_rankings = pd.DataFrame(columns=['MarianMT', 'Unicamp_T5', 'NLLB_1.3b'])

    for i in tqdm(range(len(df_src_lang))):

        # OpenAI call
        chain = prompt | chat
        english, translation_pt1, translation_pt2, translation_pt3 = create_sequences(i)
        response = chain.invoke(input={"source_seg": english,
                        "target_seg1": translation_pt1,
                        "target_seg2": translation_pt2,
                        "target_seg3": translation_pt3})
        
        total_tokens += response.response_metadata.get('token_usage').get('total_tokens')

        results = ast.literal_eval(response.content)            
        best = results.index(max(results))
        
        # Add to rankings dataset
        df_rankings.loc[len(df_rankings)] = results
        
        # Copy to final dataset
        col_list = ['context', 'question', 'answerA', 'answerB', 'answerC']
        for col in col_list:
            col_best = col + '_' + str(best+1)
            best_result = df_tgt_lang.loc[i, col_best]
            sentence_idx = col_list.index(col)
            df_output.iat[i, sentence_idx] = best_result

    print(f"Total tokens used: {total_tokens}")
    
    return df_output, df_rankings

In [95]:
df_tst.loc[0]

context     bailey was a nice person so she called the fam...
question                          What will happen to Others?
answerA                                    talk to the family
answerB                                           hate bailey
answerC                                          thank bailey
correct                                                     C
Name: 0, dtype: object

In [96]:
# Test base
df_tst_final, df_tst_rankings = get_best_translations(df_tst, df_tst_pt)

100%|██████████| 2224/2224 [23:31<00:00,  1.58it/s]

Total tokens used: 684752


In [97]:
df_tst_final

,context,question,answerA,answerB,answerC,correct
0,"bailey foi uma pessoa bonita, assim ela chamou...",O que acontecerá com os outros?,conversar com a família,baile de ódio,agradecer bailey,C
1,Alex encontrou um anel na praia e tentou desco...,O que é que a pessoa que perdeu vai querer faz...,Atira-o para o oceano .,"Transforme-o em ""perdidos e encontrados"".",Deixe a praia,B
2,Quinn jogador com remy balançando ela ao redor...,O que Quinn vai querer fazer a seguir?,Seja amigável com Remy,Tenha a confiança de Remy,Põe o Remy no chão.,C
3,"O Casey era popular no liceu, era um jogador d...",Como descreveria o Casey?,uma pessoa preguiçosa,Casey que tem uma estrela de futebol,pessoa social,C
4,Alex bateu na mesa onde o café estava colocado.,O que vai acontecer ao Riley?,Continua a beber o café.,derramar o café,Tenho de comprar um recheio.,B
...,...,...,...,...,...,...
2219,Bailey passou o fim de semana inteiro na casa ...,Como Bailey se sentiria depois?,estressada,serena,agitado,B
2220,"Jesse comeu, bebeu e conversou com os amigos a...",O que é que o Jesse vai querer fazer a seguir?,Vai para casa a seguir.,Pede mais comida a seguir.,Beba o que quisesse.,A
2221,Alex via desenhos animados e comeu um lanche d...,Porque é que a Alex fez isto?,Fazer lições de casa,Relaxa .,Tenho de pensar na escola.,B
2222,"Quinn, diz-me os filmes de que o Jordan está m...",Como descreveria o Quinn?,orgulhoso dos progressos do Jordan,grato por fazer parte da jornada do Jordan,Um escritor,A


In [98]:
df_tst_rankings

,MarianMT,Unicamp_T5,NLLB_1.3b
0,85,90,80
1,85,80,90
2,75,60,50
3,75,60,80
4,60,70,80
...,...,...,...
2219,85,75,70
2220,85,70,90
2221,65,50,75
2222,90,85,95


In [100]:
df_tst_final.to_csv('translated/final/tst.csv')
df_tst_rankings.to_csv('rankings/tst_rankings.csv')